[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/GeoAI-Tutorials/blob/main/docs/PDFM/zillow_home_value.ipynb)

**Predicting US Housing Prices at the Zip Code Level Using Google's Population Dynamics Foundation Model and Zillow Data**

## Useful Resources

- [Google's Population Dynamics Foundation Model (PDFM)](https://github.com/google-research/population-dynamics)
- Request access to PDFM embeddings [here](https://github.com/google-research/population-dynamics?tab=readme-ov-file#getting-access-to-the-embeddings)
- Zillow data can be accessed [here](https://www.zillow.com/research/data/)

## Acknowledgements

This notebook is adapted from the [PDFM tutorial](https://github.com/google-research/population-dynamics/blob/master/notebooks/pdfm_earth_engine.ipynb). Credit goes to the authors of the PDFM tutorial.

## Installation

Uncomment and run the following cell to install the required libraries.

In [ ]:
# %pip install leafmap scikit-learn

## Import Libraries

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from leafmap.common import evaluate_model, plot_actual_vs_predicted, download_file

## Download Zillow Data

In [ ]:
zhvi_url = "https://github.com/opengeos/datasets/releases/download/us/zillow_home_value_index_by_zipcode.csv"
zhvi_file = "data/zillow_home_value_index_by_zipcode.csv"

In [ ]:
if not os.path.exists(zhvi_file):
    download_file(zhvi_url, zhvi_file)

## Process Zillow Data

In [ ]:
zhvi_df = pd.read_csv(zhvi_file, dtype={"RegionName": "string"})
zhvi_df.index = zhvi_df["RegionName"].apply(lambda x: f"zip/{x}")
zhvi_df.head()

## Request access to PDFM Embeddings

In [ ]:
embeddings_file_path = "data/zcta_embeddings.csv"

To request access to PDFM embeddings, please follow the instructions [here](https://github.com/google-research/population-dynamics?tab=readme-ov-file#getting-access-to-the-embeddings).

In [ ]:
if not os.path.exists(embeddings_file_path):
    raise FileNotFoundError("Please request the embeddings from Google")

## Load PDFM Embeddings

In [ ]:
zipcode_embeddings = pd.read_csv(embeddings_file_path).set_index("place")
zipcode_embeddings.head()

## Join Zillow and PDFM Data

In [ ]:
data = zhvi_df.join(zipcode_embeddings, how="inner")
data.head()

In [ ]:
embedding_features = [f"feature{x}" for x in range(330)]
label = "2024-10-31"

In [ ]:
data = data.dropna(subset=[label])

## Split Train and Test Data

In [ ]:
data = data[embedding_features + [label]]
X = data[embedding_features]
y = data[label]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

## Fit Linear Regression Model

In [ ]:
# Initialize and train a simple linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

## Evaluate Linear Regression Model

In [ ]:
evaluation_df = pd.DataFrame({"y": y_test, "y_pred": y_pred})
metrics = evaluate_model(evaluation_df)
print(metrics)

In [ ]:
xy_lim = (0, 3_000_000)
plot_actual_vs_predicted(
    evaluation_df,
    xlim=xy_lim,
    ylim=xy_lim,
    title="Actual vs Predicted Home Values",
    x_label="Actual Home Value",
    y_label="Predicted Home Value",
)

![image](https://github.com/user-attachments/assets/286638f1-88b6-4327-a883-c4e4512fbcdb)

## Fit K-Nearest Neighbors Model

In [ ]:
k = 5
model = KNeighborsRegressor(n_neighbors=k)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

## Evaluate K-Nearest Neighbors Model

In [ ]:
evaluation_df = pd.DataFrame({"y": y_test, "y_pred": y_pred})
# Evaluate the model
metrics = evaluate_model(evaluation_df)
print(metrics)

In [ ]:
plot_actual_vs_predicted(
    evaluation_df,
    xlim=xy_lim,
    ylim=xy_lim,
    title="Actual vs Predicted Home Values",
    x_label="Actual Home Value",
    y_label="Predicted Home Value",
)

![image](https://github.com/user-attachments/assets/e8cdcd23-1945-480b-896a-49e273716bf1)